In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from mlxtend.preprocessing import minmax_scaling
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Set Color Palettes for the Notebook
# my_color_palette = ['#594346', '#212027', '#F22F08', '#8D2F23', '#561E18']
my_color_palette = ['#99B898','#FECEAB','#FF847C','#E84A5F','#2A363B'] 
sns.palplot(sns.color_palette(my_color_palette))
plot_params = {'color': '0.75',
 'style': '.-',
 'markeredgecolor': '0.25',
 'markerfacecolor': '0.25',
 'legend': True}

# Set Style
sns.set_style('whitegrid')
sns.despine(left=True, bottom=True)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading Data

In [ ]:
# Get Ready the data
dtype = {
    'store_nbr': 'category',
    'family': 'category',
    'sales': 'float32',
    'onpromotion': 'uint64',
}
store_sales = pd.read_csv(
    '../input/store-sales-time-series-forecasting/train.csv',
    dtype=dtype,
    parse_dates=['date'],
    infer_datetime_format=True,
)
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'])
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv", parse_dates=['date'])
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv", parse_dates=['date'])
holidays_events = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=['date'])
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
sample_submission = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")

## Data Cleaning - Missing Values

In [ ]:
def data_information(data, title=None):
    print('==' * 13, f'DATA HEAD {title}', '==' * 13)
    display(data.head())
    print()

    print('==' * 13, f'DATA TAIL {title}', '==' * 13)
    display(data.tail())
    print()
    
    print('==' * 13, f'THE SHAPE OF DATA {title}', '==' * 13)
    display(data.shape)
    print()
    
    print('==' * 13, f'INFO DATA {title}', '==' * 13)
    data.info()
    print()
    
    try:
        print('==' * 13, f'DESCRIPTIVE STATISTICS CATEGORICAL DATA {title}', '==' * 13)
        display(data.select_dtypes(include=['object']).describe())
        print()
    except:
        print('==' * 13, f'NOTHING {title}', '==' * 13)
        print()
    finally:
        print('==' * 13, f'DESCRIPTIVE STATISTICS NUMERICAL DATA {title}', '==' * 13)
        display(data.describe())
        print()
    
    print('==' * 13, f'THE NUMBER OF MISSING DATA PER COLUMN {title}', '==' * 13)
    print(data.isna().sum())
    print()
    
    print('==' * 13, f'Total Missing Value {title}', '==' * 13)
    total_missing_values = data.isna().sum().sum()
    total_cells = np.product(data.shape)
    
    print('Total Cells \t\t:', total_cells)
    print('Total Missing Values \t:', total_missing_values)
    print('Percentage \t\t:', total_missing_values / total_cells * 100, '%')
    
def imputation(X, y=None, strategy='mean'):
    imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)
    imputed_X = pd.DataFrame(imputer.fit_transform(X))
    imputed_X.columns = X.columns
    try:    
        imputed_y = pd.DataFrame(imputer.transform(y))
        imputed_y.columns = y.columns
    except:
        return imputed_X
    else:
        return imputed_X, imputed_y

In [ ]:
data_information(store_sales, 'Train Data')

In [ ]:
data_information(test, 'Test Data')

In [ ]:
data_information(stores, 'Stores Data')

In [ ]:
data_information(oil, 'Oil')

In [ ]:
# oil = imputation(oil, strategy='most_frequent')
# data_information(oil, 'Imputed Oil')

In [ ]:
data_information(holidays_events, 'Holidays Events')

In [ ]:
data_information(transactions, 'Transactions')

## Data Distribution

In [ ]:
sns.distplot(store_sales['onpromotion'])

In [ ]:
store_sales['onpromotion'].min()
store_sales['onpromotion'].max()
store_sales['onpromotion'].value_counts()

print(np.where(store_sales['onpromotion'] < 10)[0].shape)
print(np.where(store_sales['onpromotion'] > 10)[0].shape)
# store_sales['onpromotion'].sort_index().value_counts()[100:150]

In [ ]:
sns.distplot(store_sales.loc[np.where(store_sales['onpromotion'] < 10)[0], 'onpromotion'])

In [ ]:
sns.distplot(store_sales.loc[np.where(store_sales['onpromotion'] > 10)[0], 'onpromotion'])

In [ ]:
sns.distplot(store_sales.loc[np.where(store_sales['onpromotion'] > 100)[0], 'onpromotion'])

In [ ]:
sns.distplot(store_sales.loc[np.where(store_sales['onpromotion'] > 300)[0], 'onpromotion'])

From the Distribution Plot above, we can get, the lower of `onpromotion`, the lower frequency of data as well
From that, we must make it `Bell-Curved` shape. So that, we will use Normalization Method for this Feature (`onpromotion`)

In [ ]:
print('The Number of Data with Zero onpromotion :', 
      np.where(store_sales['onpromotion'] < 1)[0].shape[0] / 
      store_sales['onpromotion'].shape[0] * 100, '%')

In [ ]:
normalized_onpromotion = stats.boxcox(store_sales.loc[np.where(store_sales['onpromotion'] > 0)[0], 'onpromotion'])

In [ ]:
# onpromotion_positive_idx = store_sales.loc[np.where(store_sales['onpromotion'] > 0)[0], 'onpromotion'].index
def normalize_data(data):
    positive_data = data.loc[data > 0]
    normalized_data = pd.Series(stats.boxcox(positive_data)[0], index=positive_data.index)
    data[positive_data.index] = normalized_data
    return data

In [ ]:
sns.distplot(normalize_data(store_sales['onpromotion']))

In [ ]:
sns.displot(data=store_sales, x='sales', bins=30, height=10)
plt.xlabel('Total Sales')
plt.ylabel('Count')
plt.axvline(x=store_sales['sales'].mean(), color='red', ls='--', label='Mean')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(20,15))
sns.countplot(data=store_sales, y='family', 
             order=store_sales['family'].value_counts().index,
             palette=my_color_palette, orient='v')

In [ ]:
store_sales['sales'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(data=stores, x='type', palette=my_color_palette)

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data=stores, y='state', palette=my_color_palette, order=stores['state'].value_counts().index)

In [ ]:
plt.figure(figsize=(11,8))
sns.countplot(data=stores, y='city', palette=my_color_palette, order=stores['city'].value_counts().index)

In [ ]:
store_sales.head()

In [ ]:
stores.head()

In [ ]:
holidays_events

In [ ]:
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'], index_col=0)
train

In [ ]:
train = pd.merge(train, oil, on='date')
test = pd.merge(test, oil, on='date')

train = pd.merge(train, holidays_events[['date', 'type', 'transferred']], on='date')
train = pd.merge(train, stores, on='store_nbr')
train.rename(columns={'type_x':'holiday_type', 'type_y':'store_type'}, inplace=True)

train['Year'] = train.date.dt.year
train['Year-Month'] = train['date'].apply(lambda x : x.strftime('%Y-%m'))
train['Month'] = train.date.dt.month
train['Day'] = train.date.dt.day

In [ ]:
data_information(store_sales)

In [ ]:
train.groupby(['family'])['sales'].sum().sort_values(ascending=False)

In [ ]:
train.sample(3)

In [ ]:
plt.figure(figsize=(20, 15))

sns.barplot(x='sales',y='family',data=train, 
            order=train.groupby(['family'])['sales'].sum().sort_values(ascending=False).index,
            palette=my_color_palette)
plt.title('Distribution of Sales considering Product',fontweight="bold")

In [ ]:
train.groupby(['state'])['sales'].sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(20, 15))

sns.barplot(x='sales',y='state',data=train, 
            order=train.groupby(['state'])['sales'].sum().sort_values(ascending=False).index,
            palette=my_color_palette)
plt.title('Distribution of Sales considering State',fontweight="bold")
plt.ylabel('State', fontsize=18)
plt.xlabel('Sales', fontsize=16)

--------------------------------------------------------------------------------
## Time-Series with Regression Model

In [ ]:
train['Time'] = np.arange(len(train.index))
store_sales_new = train.set_index('date').to_period('D').set_index(['store_nbr', 'family'], append=True)
average_sales = store_sales_new.groupby('date').mean()['sales']

In [ ]:
display(store_sales_new)
display(average_sales)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 9))
ax.plot('Time', 'sales', data=store_sales_new, color=my_color_palette[np.random.randint(0,4)])
ax.set_ylim(ymin=0)
ax.set_xlim(xmin=0)
ax = sns.regplot('Time', 'sales', data=store_sales_new)
ax.set_title('Time Plot of Sales')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(13, 9))
ax.plot('Time', 'sales', data=store_sales_new, color=my_color_palette[np.random.randint(0,4)])
ax = sns.regplot(x='Time', y='sales', data=store_sales_new, ci=None, scatter_kws=dict(color='0.25'))
ax.set_title('Time Plot of Store Sales');

In [ ]:
tmp = train.set_index('family').loc['AUTOMOTIVE', :]
tmp1 = tmp[tmp['sales'] > 30].sort_values('date', ascending=True).reset_index()['sales']
tmp1.plot(color=my_color_palette[0])
tmp2 = tmp[tmp['sales'] < 30].sort_values('date', ascending=True).reset_index()['sales']
tmp2.plot(color=my_color_palette[np.random.randint(0,4)])

In [ ]:
tmp = train[['date', 'family', 'sales']].sort_values('date', ascending=True)
# sns.relplot()
sns.relplot(x="date", y="sales", hue="family",
            kind="line", data=tmp);

In [ ]:
average_sales.to_frame()

In [ ]:
from sklearn.linear_model import LinearRegression

df = average_sales.to_frame()

time = np.arange(average_sales.size)
df['time'] = time

X = df.loc[:, ['time']]  # features
y = df.loc[:, 'sales']  # target

linreg = LinearRegression().fit(X, y)

y_pred = pd.Series(linreg.predict(X), index=X.index)
y_pred

In [ ]:
ax = y.plot(**{'color': '0.75',
                        'style': '.-',
                        'markeredgecolor': '0.25',
                        'markerfacecolor': '0.25',
                        'legend': False}, 
            alpha=0.5)
ax = y_pred.plot(ax=ax, linewidth=3)
ax.set_title('Time Plot of total store sales');

In [ ]:
df = average_sales.to_frame()

# Create a lag feature from the target 'sales'
lag_1 = df['sales'].shift(1)

df['lag_1'] = lag_1  # add to dataframe

X = df.loc[:, ['lag_1']].dropna()  # features
y = df.loc[:, 'sales']  # target
y, X = y.align(X, join='inner')  # drop corresponding values in target

# Create a LinearRegression instance and fit it to X and y.
linreg = LinearRegression().fit(X, y)

# the same time index as the training data
y_pred = pd.Series(linreg.predict(X), index=X.index)
y_pred

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.plot(X['lag_1'], y, '.', color='0.25')
ax.plot(X['lag_1'], y_pred)
ax.set(aspect='equal', ylabel='sales', xlabel='lag_1', title='Lag Plot of Average Sales');

--------------------------------------------------------------------------------
## Trend

In [ ]:
# Identify Trend

average_sales = store_sales.set_index('date').to_period('D').set_index(['store_nbr', 'family'], append=True).groupby('date').mean()['sales']

plt.figure(figsize=(12,6))
trend = average_sales.rolling(
    window=365,
    center=True,
    min_periods=183,
).mean()

ax = average_sales.plot(**plot_params, alpha=0.5)
ax = trend.plot(ax=ax, linewidth=3)

--------------------------------------------------------------------------------
## Trend Feature

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

y = average_sales.copy()

dp = DeterministicProcess(index=y.index, order=3)
X = dp.in_sample()
X_fore = dp.out_of_sample(steps=90)

In [ ]:
plt.figure(figsize=(13,6))
model = LinearRegression()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='C0')
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='C3')
ax.legend();

--------------------------------------------------------------------------------

One way to fit more complicated trends is to increase the order of the polynomial you use. To get a better fit to the somewhat complicated trend in *Store Sales*, we could try using an order 11 polynomial.

In [ ]:
def create_trend_feature(data, order_polynomial=3):
    from statsmodels.tsa.deterministic import DeterministicProcess

    y = data.copy()

    dp = DeterministicProcess(index=y.index, order=order_polynomial)
    X = dp.in_sample()
    X_fore = dp.out_of_sample(steps=90)
    
    plt.figure(figsize=(13,6))
    model = LinearRegression()
    model.fit(X, y)

    y_pred = pd.Series(model.predict(X), index=X.index)
    y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

    ax = y.plot(**plot_params, alpha=0.5, title=f"Average Sales - {order_polynomial} Order Polynomial", ylabel="items sold")
    ax = y_pred.plot(ax=ax, linewidth=3, label="Trend", color='C0')
    ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color='C3')
    ax.legend();

In [ ]:
for i in range(1, 12):
    create_trend_feature(average_sales, i)

## The Risks of forecasting with high-order polynomials

High-order polynomials are generally not well-suited to forecasting, however. Can you guess why?
An order 11 polynomial will include terms like `t ** 11`. Terms like these tend to diverge rapidly outside of the training period making forecasts very unreliable.

--------------------------------------------------------------------------------
## Multivariate Adaptive Regression Splines (MARS) algorithm
Splines are a nice alternative to polynomials when you want to fit a trend. The Multivariate Adaptive Regression Splines (MARS) algorithm in the pyearth library is powerful and easy to use. There are a lot of hyperparameters you may want to investigate.

In [ ]:
from pyearth import Earth

# Target and features are the same as before
y = average_sales.copy()
dp = DeterministicProcess(index=y.index, order=1)
X = dp.in_sample()

# Fit a MARS model with `Earth`
model = Earth()
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=X.index)

plt.figure(figsize=(12,6))
ax = y.plot(**plot_params, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, linewidth=3, label="Trend")

Forecasting complicated trends like this will typically be difficult (if not impossible). With historical data, however, you can use splines to isolate other patterns in a time series by *detrending*.

In [ ]:
y_detrended = y - y_pred   # remove the trend from store_sales

plt.figure(figsize=(12,6))

y_detrended.plot(**plot_params, title="Detrended Average Sales");

--------------------------------------------------------------------------------
## Seasonality

In [ ]:
# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

In [ ]:
comp_dir = Path('../input/store-sales-time-series-forecasting')

holidays_events = pd.read_csv(
    comp_dir / "holidays_events.csv",
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
holidays_events = holidays_events.set_index('date').to_period('D')

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()
average_sales = (
    store_sales
    .groupby('date').mean()
    .squeeze()
    .loc['2017']
)

In [ ]:
X = average_sales.to_frame()
X["week"] = X.index.week
X["day"] = X.index.dayofweek
seasonal_plot(X, y='sales', period='week', freq='day');

In [ ]:
plot_periodogram(average_sales);

### Determine Seasonality
Both the seasonal plot and the periodogram suggest a strong weekly seasonality. From the periodogram, it appears there may be some monthly and biweekly components as well. In fact, the notes to the Store Sales dataset say wages in the public sector are paid out biweekly, on the 15th and last day of the month -- a possible origin for these seasons.

### Create seasonal features
Use `DeterministicProcess` and `CalendarFourier` to create:

+ indicators for weekly seasons and
+ Fourier features of order 4 for monthly seasons.

In [ ]:
y = average_sales.copy()

fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()

#### Fit The Seasonal Model

In [ ]:
model = LinearRegression().fit(X, y)
y_pred = pd.Series(
    model.predict(X),
    index=X.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X), index=X.index)
ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();

In [ ]:
from sklearn import metrics
print('Mean Squared Error \t:', metrics.mean_squared_error(y, y_pred))
print('Root Mean Squared Error :', np.sqrt(metrics.mean_squared_error(y, y_pred)))
print('Mean Absolute Error \t:', metrics.mean_absolute_error(y, y_pred))
print('Mean Squared Log Error \t:', metrics.mean_squared_log_error(y, y_pred))

### Deseasonalizing the series.

In [ ]:
y_deseason = y - y_pred

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(10, 7))
ax1 = plot_periodogram(y, ax=ax1)
ax1.set_title("Product Sales Frequency Components")
ax2 = plot_periodogram(y_deseason, ax=ax2);
ax2.set_title("Deseasonalized");

The periodogram for the deseasonalized series lacks any large values. By comparing it to the periodogram for the original series, we can see that our model was able to capture the seasonal variation in Average Sales.
_______________________________________________
### Ecuadorian holidays

In [ ]:
holidays = (
    holidays_events
    .query("locale in ['National', 'Regional']")
    .loc['2017':'2017-08-15', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
)

display(holidays)

From a plot of the deseasonalized *Average Sales*, it appears these holidays could have some predictive power.

In [ ]:
ax = y_deseason.plot(**plot_params)
plt.plot_date(holidays.index, y_deseason[holidays.index], color='C3')
ax.set_title('National and Regional Holidays');

### Create Holiday Feature

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse=False)

X_holidays = pd.DataFrame(
    ohe.fit_transform(holidays),
    index=holidays.index,
    columns=holidays.description.unique(),
)

X2 = X.join(X_holidays, on='date').fillna(0.0)

### Fit the seasonal model with holiday features added

In [ ]:
model = LinearRegression().fit(X2, y)
y_pred = pd.Series(
    model.predict(X2),
    index=X2.index,
    name='Fitted',
)

y_pred = pd.Series(model.predict(X2), index=X2.index)
ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax.legend();

### Create Seasonal Model for the Full *Store Sales* with all 1800 Time Series

In [ ]:
y = store_sales.unstack(['store_nbr', 'family']).loc["2017"]

# Create training data
fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()
X['NewYear'] = (X.index.dayofyear == 1)

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [ ]:
def sales_at_store(store, family):
    # Uncomment to see a list of product families
    # display(store_sales.index.get_level_values('family').unique())

    ax = y.loc(axis=1)['sales', store, family].plot(**plot_params)
    ax = y_pred.loc(axis=1)['sales', store, family].plot(ax=ax)
    ax.set_title(f'{family} Sales at Store {store}');

STORE_NBR = '1'  # 1 - 54
FAMILY = 'PRODUCE'

display(pd.Series(store_sales.index.get_level_values('family').unique())) # A list of product families

ax = y.loc(axis=1)['sales', STORE_NBR, FAMILY].plot(**plot_params)
ax = y_pred.loc(axis=1)['sales', STORE_NBR, FAMILY].plot(ax=ax)
ax.set_title(f'{FAMILY} Sales at Store {STORE_NBR}');

In [ ]:
sales_at_store('3', 'CLEANING')

## Submission 1 with Linear Regression

In [ ]:
df_test = pd.read_csv(
    comp_dir / 'test.csv',
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

# Create features for test set
X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)


y_submit = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
y_submit.to_csv('submission.csv', index=False)

## Cycles

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex4 import *

# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_lags, make_lags, make_leads

from sklearn.metrics import mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean() 
    .unstack('family')
    .loc['2017', ['sales', 'onpromotion']]
)

mag_sales = family_sales.loc(axis=1)[:, 'MAGAZINES']

----------------------------------------------------------------------------

Not every product family has sales showing cyclic behavior, and neither does the series of average sales. Sales of magazines, however, show patterns of growth and decay not well characterized by trend or seasons. In this question and the next, you'll model cycles in magazine sales using lag features.

Trend and seasonality will both create serial dependence that shows up in correlograms and lag plots. To isolate any purely *cyclic* behavior, we'll start by deseasonalizing the series. Use the code in the next cell to deseasonalize *Magazine Sales*. We'll store the result in a variable `y_deseason`.

In [ ]:
y = mag_sales.loc[:, 'sales'].squeeze()

fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    constant=True,
    index=y.index,
    order=1,
    seasonal=True,
    drop=True,
    additional_terms=[fourier],
)
X_time = dp.in_sample()
X_time['NewYearsDay'] = (X_time.index.dayofyear == 1)

model = LinearRegression(fit_intercept=False)
model.fit(X_time, y)
y_deseason = y - model.predict(X_time)
y_deseason.name = 'sales_deseasoned'

ax = y_deseason.plot(color='darkblue')
ax.set_title("Magazine Sales (deseasonalized)");

Does this deseasonalized series show cyclic patterns? To confirm our intuition, we can try to isolate cyclic behavior using a moving-average plot. The idea is to choose a window long enough to smooth over short-term seasonality, but short enough to still preserve the cycles.

### Plotting cycles

Create a seven-day moving average from `y`, the series of magazine sales. Use a centered window, but don't set the `min_periods` argument.

In [ ]:
y_ma = y.rolling(7, center=True).mean()

ax = y_ma.plot(color='green')
ax.set_title("Seven-Day Moving Average");

Do you see how the moving average plot resembles the plot of the deseasonalized series? In both, we can see cyclic behavior indicated.

-------------------------------------------------------------------------------

Let's examine our deseasonalized series for serial dependence. Take a look at the partial autocorrelation correlogram and lag plot.

In [ ]:
plot_pacf(y_deseason, lags=8);
plot_lags(y_deseason, lags=8, nrows=2);

### Serial Dependence in *Store Sales*

Are any of the lags significant according to the correlogram? Does the lag plot suggest any relationships that weren't apparent from the correlogram? <br>
The correlogram indicates the first lag is likely to be significant, as well as possibly the sixth lag. The lag plot suggests some non-linear effect as well.

-------------------------------------------------------------------------------

Recall from the tutorial that a *leading indicator* is a series whose values at one time can be used to predict the target at a future time -- a leading indicator provides "advance notice" of changes in the target.

The competition dataset includes a time series that could potentially be useful as a leading indicator -- the `onpromotion` series, which contains the number of items on a special promotion that day. Since the company itself decides when to do a promotion, there's no worry about "lookahead leakage"; we could use Tuesday's `onpromotion` value to forecast sales on Monday, for instance.

Use the next cell to examine leading and lagging values for `onpromotion` plotted against magazine sales.

In [ ]:
onpromotion = mag_sales.loc[:, 'onpromotion'].squeeze().rename('onpromotion')

# Drop the New Year outlier
plot_lags(x=onpromotion.iloc[1:], y=y_deseason.iloc[1:], lags=3, leads=3, nrows=1);

### Time Series Features
The lag plot indicates that both leading and lagged values of onpromotion are correlated with magazine sales. This suggests that both kinds of values could be useful as features.

In addition, the leading values seem to have some non-linear effect.

In [ ]:
X_lags = make_lags(y_deseason, lags=1)

X_promo = pd.concat([
    make_lags(onpromotion, lags=1),
    onpromotion,
    make_leads(onpromotion, leads=1),
], axis=1)

X_oil = pd.DataFrame()

X = pd.concat([X_time, X_lags, X_promo, X_oil], axis=1).dropna()
y, X = y.align(X, join='inner')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=30, shuffle=False)

model = LinearRegression(fit_intercept=False).fit(X_train, y_train)
y_fit = pd.Series(model.predict(X_train), index=X_train.index).clip(0.0)
y_pred = pd.Series(model.predict(X_valid), index=X_valid.index).clip(0.0)

rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_valid, y_pred) ** 0.5
print(f'Training RMSLE: {rmsle_train:.5f}')
print(f'Validation RMSLE: {rmsle_valid:.5f}')

ax = y.plot(**plot_params, alpha=0.5, title="Average Sales", ylabel="items sold")
ax = y_fit.plot(ax=ax, label="Fitted", color='C0')
ax = y_pred.plot(ax=ax, label="Forecast", color='C3')
ax.legend();

### Create statistical features

the code in the next cell will create the following features:
- 14-day rolling median (`median`) of lagged target
- 7-day rolling standard deviation (`std`) of lagged target
- 7-day sum (`sum`) of items "on promotion", with centered window

In [ ]:
y_lag = mag_sales.loc[:, 'sales'].shift(1)
onpromo = mag_sales.loc[:, 'onpromotion']

mean_7 = y_lag.rolling(7).mean()
median_14 = y_lag.rolling(14).median()
std_7 = y_lag.rolling(7).std()
promo_7 = onpromo.rolling(7, center=True).sum()

## Hybrid Models

In [ ]:
# Setup notebook

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

# Class for Hybrid Models
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None  # store column names from fit method

In [ ]:
def fit(self, X_1, X_2, y):
    # fit self.model_1
    self.model_1.fit(X_1, y)

    y_fit = pd.DataFrame(
        # make predictions with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=y.columns,
    )

    # compute residuals
    y_resid = y - y_fit
    y_resid = y_resid.stack().squeeze() # wide to long

    # fit self.model_2 on residuals
    self.model_2.fit(X_2, y_resid)

    # Save column names for predict method
    self.y_columns = y.columns
    # Save data for question checking
    self.y_fit = y_fit
    self.y_resid = y_resid


# Add method to class
BoostedHybrid.fit = fit

def predict(self, X_1, X_2):
    y_pred = pd.DataFrame(
        # predict with self.model_1
        self.model_1.predict(X_1),
        index=X_1.index, columns=self.y_columns,
    )
    y_pred = y_pred.stack().squeeze()  # wide to long

    # add self.model_2 predictions to y_pred
    y_pred += self.model_2.predict(X_2)
    
    return y_pred.unstack()  # long to wide


# Add method to class
BoostedHybrid.predict = predict


### Set Up data for Training

In [ ]:
# Target series
y = family_sales.loc[:, 'sales']


# X_1: Features for Linear Regression
dp = DeterministicProcess(index=y.index, order=1)
X_1 = dp.in_sample()


# X_2: Features for XGBoost
X_2 = family_sales.drop('sales', axis=1).stack()  # onpromotion feature

# Label encoding for 'family'
le = LabelEncoder()
X_2 = X_2.reset_index('family')
X_2['family'] = le.fit_transform(X_2['family'])

# Label encoding for seasonality
X_2["day"] = X_2.index.day  # values are day of the month

### Train boosted hybrid

Create the hybrid model by initializing a `BoostedHybrid` class with `LinearRegression()` and `XGBRegressor()` instances.

In [ ]:
# Create LinearRegression + XGBRegressor hybrid with BoostedHybrid
model = BoostedHybrid(model_1=LinearRegression(), model_2=XGBRegressor())

# Fit and predict
model.fit(X_1, X_2, y)
y_pred = model.predict(X_1, X_2)

y_pred = y_pred.clip(0.0)

### Try Other Algorithms

In [ ]:
# Model 1 (trend)
from pyearth import Earth
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Model 2
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

# Boosted Hybrid

# Try different combinations of the algorithms above
model = BoostedHybrid(
    model_1=Ridge(),
    model_2=KNeighborsRegressor(),
)

In [ ]:
y_train, y_valid = y[:"2017-07-01"], y["2017-07-02":]
X1_train, X1_valid = X_1[: "2017-07-01"], X_1["2017-07-02" :]
X2_train, X2_valid = X_2.loc[:"2017-07-01"], X_2.loc["2017-07-02":]

# Some of the algorithms above do best with certain kinds of
# preprocessing on the features (like standardization), but this is
# just a demo.
model.fit(X1_train, X2_train, y_train)
y_fit = model.predict(X1_train, X2_train).clip(0.0)
y_pred = model.predict(X1_valid, X2_valid).clip(0.0)

families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C0', ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True, sharex=True, color='C3', ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

In [ ]:
# y_submit = pd.DataFrame(y_pred, index=X_test.index, columns=y.columns)
# y_submit = y_submit.stack(['family'])
# y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])
# y_submit.to_csv('submission1.csv', index=False)

In [ ]:
X_1_test = dp.out_of_sample(12)

# test_copy = test.set_index('date')
# test_copy['family'] = le.transform(test_copy['family'])
# test_copy['day'] = test_copy.index.day
# test_copy = test_copy[['family','onpromotion', 'day']]

X_2_test = test.groupby(['date', 'family'])['id', 'onpromotion'].sum().to_frame().reset_index('family')
X_2_test['family'] = le.fit_transform(X_2_test['family'])
X_2_test["day"] = X_2_test.index.day

y_pred = model.predict(X_1_test, X_2_test)
y_pred

In [ ]:
print(X_1_test.shape)
print(X_2_test.shape)
print(y_pred.shape)
print(y_pred.size)

In [ ]:
print(X_1.shape)
print(X_2.shape)
print(227 * 7491)

In [ ]:
21384 / (12*33)

In [ ]:
test.groupby(['date', 'family'])['onpromotion'].sum().to_frame().reset_index('family')

In [ ]:
family_sales.drop('sales', axis=1).stack().reset_index('family')

In [ ]:
X_2_test = test.groupby(['date', 'family'])['id', 'onpromotion'].sum().reset_index('family')
X_2_test['family'] = le.fit_transform(X_2_test['family'])
X_2_test["day"] = X_2_test.index.day
X_2_test